# Parameters

In [1]:
batch_size     = 8
learning_rate  = 0.0001
epochs         = 5

# Define model

In [2]:
from model import DepthEstimate

model = DepthEstimate()

Base model loaded DenseNet169

Model created.


# Data loader

In [3]:
from data import DataLoader

dl = DataLoader()
train_generator = dl.get_batched_dataset(batch_size)

# Get the validation dataset
validation_dataset = dl.get_validation_dataset(batch_size)

print('Data loader ready.')

nyu2_train :  [['data/nyu2_train/living_room_0038_out/37.jpg', 'data/nyu2_train/living_room_0038_out/37.png'], ['data/nyu2_train/living_room_0038_out/115.jpg', 'data/nyu2_train/living_room_0038_out/115.png'], ['data/nyu2_train/living_room_0038_out/6.jpg', 'data/nyu2_train/living_room_0038_out/6.png'], ['data/nyu2_train/living_room_0038_out/49.jpg', 'data/nyu2_train/living_room_0038_out/49.png'], ['data/nyu2_train/living_room_0038_out/152.jpg', 'data/nyu2_train/living_room_0038_out/152.png'], ['data/nyu2_train/living_room_0038_out/142.jpg', 'data/nyu2_train/living_room_0038_out/142.png'], ['data/nyu2_train/living_room_0038_out/94.jpg', 'data/nyu2_train/living_room_0038_out/94.png'], ['data/nyu2_train/living_room_0038_out/158.jpg', 'data/nyu2_train/living_room_0038_out/158.png'], ['data/nyu2_train/living_room_0038_out/18.jpg', 'data/nyu2_train/living_room_0038_out/18.png'], ['data/nyu2_train/living_room_0038_out/167.jpg', 'data/nyu2_train/living_room_0038_out/167.png'], ['data/nyu2_train

# Compile & Train

In [4]:
import tensorflow
from loss import depth_loss_function

optimizer = tensorflow.keras.optimizers.legacy.Adam(lr=learning_rate, amsgrad=True)

# model.compile(loss=depth_loss_function, optimizer=optimizer)

/Users/adeelhafeez/GPU_Test/env/lib/python3.10/site-packages/keras/src/optimizers/legacy/adam.py:118: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [5]:
# Create checkpoint callback
import os
checkpoint_path = "training_11/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback = tensorflow.keras.callbacks.ModelCheckpoint(checkpoint_path, save_weights_only=True, save_best_only=True, verbose=1)


In [6]:
import os
import tensorflow as tf
from itertools import product

# Reduced weight range
weight_range = [0.0, 0.5, 1.0]

# Initialize the DataLoader
dl = DataLoader(DEBUG=False)

# Placeholder for best validation loss and corresponding weights
best_val_loss = float('inf')
best_weights = None

# Loop over all possible combinations of weights
for w1, w2, w3 in product(weight_range, repeat=3):
    # Create a new instance of the model for each combination of weights
    model = DepthEstimate()  # Initialize the model

    # Compile the model with the current weights
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

    def custom_loss(y_true, y_pred):
        global w1, w2, w3  # Add these lines to access the global weights
        return depth_loss_function(y_true, y_pred, theta=0.1, maxDepthVal=1000.0/10.0)

    model.compile(loss=custom_loss, optimizer=optimizer)

    # Create training and validation datasets using your DataLoader class
    train_dataset = dl.get_batched_dataset(batch_size)
    validation_dataset = dl.get_validation_dataset(batch_size)

    # Calculate the number of steps per epoch based on the dataset size and batch size
    steps_per_epoch = dl.length // batch_size

    # Inside your training loop
    history = model.fit(
        train_dataset,
        epochs=5,  # Train for 5 epochs
        steps_per_epoch=steps_per_epoch,  # Specify the number of steps
        verbose=1
    )

    # Calculate the average validation loss after training
    val_loss = model.evaluate(validation_dataset, verbose=0)

    # Check if this combination of weights results in a lower validation loss
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_weights = (w1, w2, w3)

# Print the best weights for this combination
print(f"Best Weights (w1, w2, w3): {best_weights}")
print(f"Best Validation Loss: {best_val_loss}")


nyu2_train :  [['data/nyu2_train/living_room_0038_out/37.jpg', 'data/nyu2_train/living_room_0038_out/37.png'], ['data/nyu2_train/living_room_0038_out/115.jpg', 'data/nyu2_train/living_room_0038_out/115.png'], ['data/nyu2_train/living_room_0038_out/6.jpg', 'data/nyu2_train/living_room_0038_out/6.png'], ['data/nyu2_train/living_room_0038_out/49.jpg', 'data/nyu2_train/living_room_0038_out/49.png'], ['data/nyu2_train/living_room_0038_out/152.jpg', 'data/nyu2_train/living_room_0038_out/152.png'], ['data/nyu2_train/living_room_0038_out/142.jpg', 'data/nyu2_train/living_room_0038_out/142.png'], ['data/nyu2_train/living_room_0038_out/94.jpg', 'data/nyu2_train/living_room_0038_out/94.png'], ['data/nyu2_train/living_room_0038_out/158.jpg', 'data/nyu2_train/living_room_0038_out/158.png'], ['data/nyu2_train/living_room_0038_out/18.jpg', 'data/nyu2_train/living_room_0038_out/18.png'], ['data/nyu2_train/living_room_0038_out/167.jpg', 'data/nyu2_train/living_room_0038_out/167.png'], ['data/nyu2_train

Base model loaded DenseNet169

Model created.
Epoch 1/5
43/43 [==============================] - 89s 2s/step - loss: 1.1030
Epoch 2/5
43/43 [==============================] - 75s 2s/step - loss: 0.5638
Epoch 3/5
43/43 [==============================] - 76s 2s/step - loss: 0.4622
Epoch 4/5
43/43 [==============================] - 75s 2s/step - loss: 0.4666
Epoch 5/5
43/43 [==============================] - 77s 2s/step - loss: 0.4476


Base model loaded DenseNet169

Model created.
Epoch 1/5
43/43 [==============================] - 98s 2s/step - loss: 1.0640
Epoch 2/5
43/43 [==============================] - 75s 2s/step - loss: 0.5449
Epoch 3/5
43/43 [==============================] - 74s 2s/step - loss: 0.5576
Epoch 4/5
43/43 [==============================] - 75s 2s/step - loss: 0.4294
Epoch 5/5
43/43 [==============================] - 75s 2s/step - loss: 0.3591


Base model loaded DenseNet169

Model created.
Epoch 1/5
43/43 [==============================] - 109s 2s/step - loss: 1.1072
Epoch 2/5
43/43 [==============================] - 79s 2s/step - loss: 0.5440
Epoch 3/5
43/43 [==============================] - 78s 2s/step - loss: 0.5601
Epoch 4/5
43/43 [==============================] - 78s 2s/step - loss: 0.4554
Epoch 5/5
43/43 [==============================] - 78s 2s/step - loss: 0.4305


Base model loaded DenseNet169

Model created.
Epoch 1/5
43/43 [==============================] - 126s 2s/step - loss: 1.0071
Epoch 2/5
43/43 [==============================] - 82s 2s/step - loss: 0.7244
Epoch 3/5
43/43 [==============================] - 80s 2s/step - loss: 0.4989
Epoch 4/5
43/43 [==============================] - 78s 2s/step - loss: 0.4446
Epoch 5/5
43/43 [==============================] - 78s 2s/step - loss: 0.3860


...

Base model loaded DenseNet169

Model created.
Epoch 1/5
43/43 [==============================] - 299s 5s/step - loss: 1.0841
Epoch 2/5
43/43 [==============================] - 99s 2s/step - loss: 0.8075
Epoch 3/5
43/43 [==============================] - 97s 2s/step - loss: 0.5680
Epoch 4/5
43/43 [==============================] - 100s 2s/step - loss: 0.4750
Epoch 5/5
43/43 [==============================] - 97s 2s/step - loss: 0.3964


Base model loaded DenseNet169

Model created.
Epoch 1/5
43/43 [==============================] - 278s 5s/step - loss: 1.0584
Epoch 2/5
43/43 [==============================] - 96s 2s/step - loss: 0.5495
Epoch 3/5
43/43 [==============================] - 95s 2s/step - loss: 0.5189
Epoch 4/5
43/43 [==============================] - 95s 2s/step - loss: 0.4802
Epoch 5/5
43/43 [==============================] - 94s 2s/step - loss: 0.3835


Base model loaded DenseNet169

Model created.
Epoch 1/5
43/43 [==============================] - 293s 5s/step - loss: 1.0678
Epoch 2/5
43/43 [==============================] - 99s 2s/step - loss: 0.6863
Epoch 3/5
43/43 [==============================] - 101s 2s/step - loss: 0.7125
Epoch 4/5
43/43 [==============================] - 98s 2s/step - loss: 0.4437
Epoch 5/5
43/43 [==============================] - 96s 2s/step - loss: 0.3870


Base model loaded DenseNet169

Model created.
Epoch 1/5
43/43 [==============================] - 300s 5s/step - loss: 1.1973
Epoch 2/5
43/43 [==============================] - 107s 2s/step - loss: 0.5722
Epoch 3/5
43/43 [==============================] - 98s 2s/step - loss: 0.4901
Epoch 4/5
43/43 [==============================] - 98s 2s/step - loss: 0.4526
Epoch 5/5
43/43 [==============================] - 97s 2s/step - loss: 0.4356


Base model loaded DenseNet169

Model created.
Epoch 1/5
43/43 [==============================] - 321s 6s/step - loss: 1.1007
Epoch 2/5
43/43 [==============================] - 103s 2s/step - loss: 0.5935
Epoch 3/5
43/43 [==============================] - 99s 2s/step - loss: 0.5202
Epoch 4/5
43/43 [==============================] - 99s 2s/step - loss: 0.3849
Epoch 5/5
43/43 [==============================] - 97s 2s/step - loss: 0.4103
Best Weights (w1, w2, w3): (1.0, 0.5, 1.0)
Best Validation Loss: 0.3698512613773346


In [7]:
print("Best Weights (w1, w2, w3):", best_weights)
print("Best Validation Loss:", best_val_loss)

Best Weights (w1, w2, w3): (1.0, 0.5, 1.0)
Best Validation Loss: 0.3698512613773346
